# SQLAlchemy官方示例

[参考链接](https://docs.sqlalchemy.org/en/20/orm/quickstart.html#declare-models)

## 创建数据模型

In [27]:
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class Base(DeclarativeBase):
    pass

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(
        back_populates="user", cascade="all, delete-orphan"
    )
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    user: Mapped["User"] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

## 创建连接

In [28]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///test2.db", echo=True)

## 创建数据表

In [29]:
Base.metadata.create_all(engine)

2025-02-21 17:31:12,864 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-21 17:31:12,865 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2025-02-21 17:31:12,866 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-21 17:31:12,868 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2025-02-21 17:31:12,868 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-21 17:31:12,869 INFO sqlalchemy.engine.Engine COMMIT


## 插入数据

In [30]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")],
    )
    sandy = User(
        name="sandy",
        fullname="Sandy Cheeks",
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ],
    )
    patrick = User(name="patrick", fullname="Patrick Star")
    session.add_all([spongebob, sandy, patrick])
    session.commit()

2025-02-21 17:31:24,626 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-21 17:31:24,678 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2025-02-21 17:31:24,679 INFO sqlalchemy.engine.Engine [generated in 0.00105s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('spongebob', 'Spongebob Squarepants')
2025-02-21 17:31:24,680 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2025-02-21 17:31:24,681 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/3 (ordered; batch not supported)] ('sandy', 'Sandy Cheeks')
2025-02-21 17:31:24,681 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2025-02-21 17:31:24,682 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/3 (ordered; batch not supported)] ('patrick', 'Patrick Star')
2025-02-21 17:31:24,687 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?) RETURN

## 查询数据

In [34]:
from sqlalchemy import select

session = Session(engine)

stmt = select(User).where(User.name.in_(["spongebob", "sandy"]))

for user in session.scalars(stmt):
    print(user)

2025-02-21 17:42:58,887 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-21 17:42:58,892 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name IN (?, ?)
2025-02-21 17:42:58,894 INFO sqlalchemy.engine.Engine [cached since 669.1s ago] ('spongebob', 'sandy')
User(id=1, name='spongebob', fullname='Spongebob Squarepants')
User(id=2, name='sandy', fullname='Sandy Cheeks')


## join查询

In [32]:
stmt = (
    select(Address)
    .join(Address.user)
    .where(User.name == "sandy")
    .where(Address.email_address == "sandy@sqlalchemy.org")
)
sandy_address = session.scalars(stmt).one()
print(sandy_address)

2025-02-21 17:32:22,397 INFO sqlalchemy.engine.Engine SELECT address.id, address.email_address, address.user_id 
FROM address JOIN user_account ON user_account.id = address.user_id 
WHERE user_account.name = ? AND address.email_address = ?
2025-02-21 17:32:22,401 INFO sqlalchemy.engine.Engine [generated in 0.00525s] ('sandy', 'sandy@sqlalchemy.org')
Address(id=2, email_address='sandy@sqlalchemy.org')


## 修改

In [33]:
stmt = select(User).where(User.name == "patrick")
patrick = session.scalars(stmt).one()
patrick.addresses.append(Address(email_address="patrickstar@sqlalchemy.org"))
sandy_address.email_address = "sandy_cheeks@sqlalchemy.org"

session.commit()

2025-02-21 17:42:49,995 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2025-02-21 17:42:49,997 INFO sqlalchemy.engine.Engine [generated in 0.00291s] ('patrick',)
2025-02-21 17:42:50,066 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2025-02-21 17:42:50,071 INFO sqlalchemy.engine.Engine [generated in 0.00523s] (3,)
2025-02-21 17:42:50,120 INFO sqlalchemy.engine.Engine UPDATE address SET email_address=? WHERE address.id = ?
2025-02-21 17:42:50,121 INFO sqlalchemy.engine.Engine [generated in 0.00128s] ('sandy_cheeks@sqlalchemy.org', 2)
2025-02-21 17:42:50,167 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2025-02-21 17:42:50,167 INFO sqlalchemy.engine.Engine [generated in 0.00083s] ('patrickstar@sqlalchemy.org', 3)


## 删除

In [ ]:
sandy = session.get(User, 2)
sandy.addresses.remove(sandy_address)
session.flush()
session.delete(patrick)
session.commit()